# Identify and examine labels to find safe-for-you beauty products

In [290]:
import pandas as pd
import numpy as np
from sklearn import base
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.neighbors import NearestNeighbors
import re
import spacy
from sklearn.decomposition import TruncatedSVD
from spacy.lang.en.stop_words import STOP_WORDS

In [291]:
#1.user input a skincare product
#2.user identify an allergen/irritation
#3.recommend products

In [292]:
#conda install -c conda-forge spacy
#python -m spacy download en
#pip install streamlit-extras

In [390]:
ulta=pd.read_csv('Clean_data')
ulta.sample(1)

,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Main_Category,Category,Details,Image
3257,My Clarins,Re-Boost Matifying Hydrating Cream,5.0,$27.00,https://www.ulta.com/p/re-boost-matifying-hydr...,2598902,2.0,"Aqua/Water/Eau, Cetearyl Isononanoate, Glyceri...",Skincare,Moisturizers,Details\nBenefits\nDaily matte gel moisturizer...,https://media.ulta.com/i/ulta/2598902?w=78&h=78


In [391]:
ulta['Ingredient'][3257]

'Aqua/Water/Eau, Cetearyl Isononanoate, Glycerin, Propanediol, Cetearyl Alcohol, 1,2- Hexanediol, Cocos Nucifera (Coconut) Fruit Juice, Butylene Glycol, Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer, Cetearyl Glucoside, Glyceryl Stearate, Peg-100 Stearate, Propylene Glycol, Xylitylglucoside, Dimethicone, Cellulose, Tapioca Starch, Anhydroxylitol, Ethylhexylglycerin, Parfum/Fragrance, Sodium Polyacrylate, Silica Silylate, Xanthan Gum, Xylitol, Caprylic/Capric Triglyceride, Tartaric Acid, Citric Acid, Glucose, Tamarindus Indica Extract, Spiraea Ulmaria Extract, Marrubium Vulgare Extract, Lycium Barbarum Fruit Extract, Ficus Carica (Fig) Fruit Extract, Malpighia Emarginata (Acerola) Seed Extract, Phenethyl Alcohol, Furcellaria Lumbricalis Extract, Sodium Benzoate, Rhododendron Ferrugineum Extract, Arbutus Unedo Fruit Extract, Potassium Sorbate, Sorbic Acid, Lapsana Communis Flower/Leaf/Stem Extract, Maris Sal/Sea Salt/Sel Marin, Ci 14700/Red 4, V3148B.'

In [286]:
#Check if Ingredient list contains allergen item

def containsAllergen(df,allergen):
    return df[ df['Ingredient'].str.contains(allergen, na=False,case=False)==False]

## Model Selection:

### Model 1: CountVectorizer, NearestNeighbors

In [405]:
%%time
#CountVectorizer,NearestNeighbors
#based on ingredients of previously working-well products
#check products for problematic ingredients

def recommender_1(df,product, allergen):
    safe_df= df.copy()
    
    bag_of_words_vectorizer=CountVectorizer(min_df=0,
                             ngram_range=(1,2), 
                             stop_words='english')
    counts=bag_of_words_vectorizer.fit_transform(safe_df['Ingredient'])
    nn = NearestNeighbors(n_neighbors=20).fit(counts) 
    index=safe_df[safe_df['Name']==product].index
    print(index)
    #safe_df.index[safe_df['Name']==product][0]
    dists, indices = nn.kneighbors(counts[index[0]])
    prod_nbr=safe_df.iloc[indices[0]]
    
    prod_nbr=prod_nbr[prod_nbr['Main_Category']==df['Main_Category'][index[0]]]
    return containsAllergen(prod_nbr,allergen).head(5)
    #return #.reset_index(drop=True)

prod_1=recommender_1(ulta,'Re-Boost Matifying Hydrating Cream','Glucose')
prod_1

Int64Index([3257], dtype='int64')
CPU times: user 1.15 s, sys: 127 ms, total: 1.27 s
Wall time: 1.45 s


,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Main_Category,Category,Details,Image
1514,Clarins,Extra-Firming & Smoothing Night Moisturizer,4.6,$98.00,https://www.ulta.com/p/extra-firming-smoothing...,2592833,42.0,"Aqua/Water/Eau, Caprylic/Capric Triglyceride, ...",Skincare,Moisturizers,Details\nBenefits\nSilky moisturizer works ove...,https://media.ulta.com/i/ulta/2592833?w=156&h=156
2281,Clarins,Hydra-Essentiel Silky Moisturizer with Double ...,4.2,$49.00,https://www.ulta.com/p/hydra-essentiel-silky-m...,2606367,5.0,"Aqua/Water/Eau, Caprylic/Capric Triglyceride, ...",Skincare,Moisturizers,Details\nBenefits\nPlumper skin in 60 seconds*...,https://media.ulta.com/i/ulta/2606367?w=78&h=78
3318,My Clarins,Re-Fresh Roll-On Eye De-Puffer,3.4,$21.00,https://www.ulta.com/p/re-fresh-roll-on-eye-de...,2574883,21.0,"Aqua/Water/Eau, Butylene Glycol, Propanediol, ...",Skincare,Eye Treatments,Details\nBenefits\nVegan Formula\nContains 95%...,https://media.ulta.com/i/ulta/2574883?w=78&h=78
98,Clarins,Bright Plus Serum,4.3,$84.00,https://www.ulta.com/p/bright-plus-serum-pimpr...,2588630,240.0,"Aqua/Water/Eau, Glycerin, Ascorbyl Glucoside, ...",Skincare,Treatment & Serums,"Details\nBenefits\nVisibly tones, revitalizes ...",https://media.ulta.com/i/ulta/2588630?w=156&h=156
4006,My Clarins,RE-FRESH Hydrating Beauty Mist,4.3,$21.00,https://www.ulta.com/p/re-fresh-hydrating-beau...,2541800,586.0,"Aqua/Water/Eau, Butylene Glycol, Glycereth-26,...",Skincare,Moisturizers,Details\nBenefits\nRefreshes and hydrates\nPro...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."


### Model 2: CountVectorizer, Cosine similarity

In [412]:
%%time
#CountVectorizer,cosine similarity
"""Calculate similarity scores: Use a content-based filtering algorithm to calculate similarity scores 
between products based on their ingredient lists."""

def cos_similarity(row1, row2):
    dot = np.dot(row1, row2.T)[0, 0]
    len1 = np.sqrt(np.dot(row1, row1.T))[0, 0]
    len2 = np.sqrt(np.dot(row2, row2.T))[0, 0]
    return dot / len1 / len2


def recommender_2(df,product, allergen):
    bag_of_words_vectorizer=CountVectorizer(min_df=0,
                             ngram_range=(1,2), 
                             stop_words='english')
    counts=bag_of_words_vectorizer.fit_transform(df['Ingredient'])
    #print(counts.shape)
    prod_nbr={}
    for j in range(len(counts.toarray())):
        prod_nbr[j]=cos_similarity(counts[2147],counts[j])
    nbr=sorted(prod_nbr.items(),key=lambda x:x[1],reverse=True )
    indices=[x[0] for x in nbr[0:30]]

    prod_nbr=df.iloc[indices] 
    index=df.index[df['Name']==product][0]
    prod_nbr=prod_nbr[prod_nbr['Main_Category']==df['Main_Category'][index]]
    return containsAllergen(prod_nbr,allergen).head(5)

prod_2=recommender_2(ulta,'Re-Boost Matifying Hydrating Cream','Glucose')
prod_2

CPU times: user 9.1 s, sys: 649 ms, total: 9.75 s
Wall time: 9.94 s


,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Main_Category,Category,Details,Image
2147,Pacifica,Glow Baby Vitamin C Brightening Facial Mask,5.0,$4.00,https://www.ulta.com/p/glow-baby-vitamin-c-bri...,2605509,1.0,"Aqua (Eau), Aloe Barbadensis Leaf Extract, Gly...",Skincare,Treatment & Serums,Details\nBenefits\nGood for dullness & dark sp...,https://media.ulta.com/i/ulta/2605509?w=78&h=78
2719,Pacifica,Vegan Collagen Hydro-Treatment Eye & Smile Lin...,2.0,$8.00,https://www.ulta.com/p/vegan-collagen-hydro-tr...,2605490,1.0,"Aqua, Glycerin, Sr-Hydrozoan Polypeptide 1 (Ve...",Skincare,Eye Treatments,Details\nBenefits\nTargeted treatment to help ...,https://media.ulta.com/i/ulta/2605490?w=78&h=78
1924,Pacifica,Vegan Collagen Hydrate & Plump Face Mask,2.0,$5.00,https://www.ulta.com/p/vegan-collagen-hydrate-...,2605508,1.0,"Aqua (Eau), Glycerin, Phenoxyethanol, Betaine,...",Skincare,Treatment & Serums,Details\nBenefits\nAll skin types especially t...,https://media.ulta.com/i/ulta/2605508?w=78&h=78
2378,Pacifica,Glow Baby Vitamin C 3-Piece Trial Kit,5.0,$20.00,https://www.ulta.com/p/glow-baby-vitamin-c-3-p...,2597448,1.0,"""Brightening Face Wash: Aqua (Eau), Cocamidopr...",Skincare,By Price,Details\nBenefits\nFresh orange-scent\nPerfect...,https://media.ulta.com/i/ulta/2597448?w=78&h=78
3276,Pacifica,Bright Stars for Glowing Skin - Vitamin C Dail...,4.3,$15.00,https://www.ulta.com/p/bright-stars-glowing-sk...,2585827,7.0,Eye Bright Undereye Vitamin C Spot Serum Mask:...,Skincare,By Price,Details\nBenefits\nVitamin C-powered boost wit...,https://media.ulta.com/i/ulta/2585827?w=78&h=78


### Model 3: DictVectorizer, NearestNeighbors

In [413]:
%%time
#Feature engineering
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col):
        self.col = col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        def to_dict(l):
            try:
                return {x.strip(): 1 for x in l}
            except TypeError:
                return {}        
        return X[self.col].apply(to_dict)


def recommender_3(df,product, allergen):
    df_1=df.copy()
    df_1['Ig_list']=df.apply(lambda x:x['Ingredient'].split(','),axis=1)
    ig_pipe = Pipeline([('encoder', DictEncoder('Ig_list')),
                     ('vectorizer', DictVectorizer())#,('svd', TruncatedSVD(n_components=300))
                   ])
    #dt_pipe=Pipeline([("encoder",DictEncoder('Dt_list')),('vectorizer',DictVectorizer())])
    #union=FeatureUnion([('Ingredient',ig_pipe),('Detail',dt_pipe)])
    features = ig_pipe.fit_transform(df_1)
    nn = NearestNeighbors(n_neighbors=20).fit(features)
    index=df_1.index[df_1['Name']==product][0]
    dists, indices = nn.kneighbors(features[index])
    
    prod_nbr=df_1.iloc[indices[0]]    
    prod_nbr=prod_nbr[prod_nbr['Main_Category']==df_1['Main_Category'][index]]
    
   
    return containsAllergen(prod_nbr,allergen).head(5)

prod_3=recommender_3(ulta,'Re-Boost Matifying Hydrating Cream','Glucose')
prod_3

CPU times: user 597 ms, sys: 586 ms, total: 1.18 s
Wall time: 1.71 s


,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Main_Category,Category,Details,Image,Ig_list
3979,My Clarins,RE-FRESH Hydrating Beauty Mist,4.3,$21.00,https://www.ulta.com/p/re-fresh-hydrating-beau...,2541800,586.0,"Aqua/Water/Eau, Butylene Glycol, Glycereth-26,...",Skincare,Moisturizers,Details\nBenefits\nRefreshes and hydrates\nPro...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","[Aqua/Water/Eau, Butylene Glycol, Glycereth-..."
1503,Clarins,Extra-Firming & Smoothing Night Moisturizer,4.6,$98.00,https://www.ulta.com/p/extra-firming-smoothing...,2592833,42.0,"Aqua/Water/Eau, Caprylic/Capric Triglyceride, ...",Skincare,Moisturizers,Details\nBenefits\nSilky moisturizer works ove...,https://media.ulta.com/i/ulta/2592833?w=156&h=156,"[Aqua/Water/Eau, Caprylic/Capric Triglyceride..."
3296,My Clarins,Re-Fresh Roll-On Eye De-Puffer,3.4,$21.00,https://www.ulta.com/p/re-fresh-roll-on-eye-de...,2574883,21.0,"Aqua/Water/Eau, Butylene Glycol, Propanediol, ...",Skincare,Eye Treatments,Details\nBenefits\nVegan Formula\nContains 95%...,https://media.ulta.com/i/ulta/2574883?w=78&h=78,"[Aqua/Water/Eau, Butylene Glycol, Propanedio..."
2265,Clarins,Hydra-Essentiel Silky Moisturizer with Double ...,4.2,$49.00,https://www.ulta.com/p/hydra-essentiel-silky-m...,2606367,5.0,"Aqua/Water/Eau, Caprylic/Capric Triglyceride, ...",Skincare,Moisturizers,Details\nBenefits\nPlumper skin in 60 seconds*...,https://media.ulta.com/i/ulta/2606367?w=78&h=78,"[Aqua/Water/Eau, Caprylic/Capric Triglyceride..."
3543,CoverGirl,Simply Ageless Skin Eye Lift Serum,3.7,$19.99,https://www.ulta.com/p/simply-ageless-skin-eye...,2566116,431.0,"Aqua/Water/Eau,Butylene Glycol, Polyurethane-3...",Skincare,Eye Treatments,Details\nBenefits\nReduces wrinkles: Eye lifti...,https://media.ulta.com/i/ulta/2566116?w=156&h=156,"[Aqua/Water/Eau, Butylene Glycol, Polyurethan..."


In [414]:
prod_1.index,prod_2.index,prod_3.index

(Int64Index([1514, 2281, 3318, 98, 4006], dtype='int64'),
 Int64Index([2147, 2719, 1924, 2378, 3276], dtype='int64'),
 Int64Index([3979, 1503, 3296, 2265, 3543], dtype='int64'))

In [415]:
prod_1['Name'],prod_2['Name'],prod_3['Name']

(1514          Extra-Firming & Smoothing Night Moisturizer
 2281    Hydra-Essentiel Silky Moisturizer with Double ...
 3318                       Re-Fresh Roll-On Eye De-Puffer
 98                                      Bright Plus Serum
 4006                       RE-FRESH Hydrating Beauty Mist
 Name: Name, dtype: object,
 2147          Glow Baby Vitamin C Brightening Facial Mask
 2719    Vegan Collagen Hydro-Treatment Eye & Smile Lin...
 1924             Vegan Collagen Hydrate & Plump Face Mask
 2378                Glow Baby Vitamin C 3-Piece Trial Kit
 3276    Bright Stars for Glowing Skin - Vitamin C Dail...
 Name: Name, dtype: object,
 3979                       RE-FRESH Hydrating Beauty Mist
 1503          Extra-Firming & Smoothing Night Moisturizer
 3296                       Re-Fresh Roll-On Eye De-Puffer
 2265    Hydra-Essentiel Silky Moisturizer with Double ...
 3543                   Simply Ageless Skin Eye Lift Serum
 Name: Name, dtype: object)

Model 3 generates the best result with the fastest speed.